# **LAB ONE: NMR SIGNAL**

Brief description of lab

#### Learning Objectives:
- Learn about magnetic fields in NMR
- Learn what NMR is
- Learn how to excite an NMR signal in a sample
- Learn how to measure the NMR signal
- Learn what causes NMR signal decay
- Learn how to optimise amplitude and duration
- Learn what an NMR spin echo is

#### Assumed Knowledge:
- LC Resonance 
- Solenoid Coils
- Fourier Transform
- Angular Momentum 
- Complex Numbers


-------------------------------------------------------------------------------------------------------------------------------------------------------
#### **Setup Task:**

To initialise this notebook, edit the cell above to set `LAB_USER_NAME` to your name, then click **Run->Run All Cells** in the top menu bar  




In [ ]:
LAB_USER_NAME = 'REPLACE_ME'

In [ ]:
import panel as pn
pn.extension()
import sys
import os
sys.path.append('../../../dashboards-inline')
LAB_DIR = os.path.join('/home/data/', LAB_USER_NAME)
os.makedirs(LAB_DIR, exist_ok=True)
print('Data will be saved to', LAB_DIR)

-------------------------------------------------------------------------------------------------------------------------------------------------------

## 1. Introduction to Nuclear Magnetic Resonance
### 1.1 Theory

<figure style="float: right;">
<img src="Images/PrecessionDiagram.png" width="200">
<figcaption style="width: 200px;">Figure 1: Hydrogen nucleus precessing in a magnetic field</figcaption>
</figure>

Nuclear Magnetic Resonance occurs when a magnetic nucleus (e.g. hydrogen) is placed in an external magnetic field. Using a classical physics model, the nucleus can be thought of as a tiny bar magnet with an orientation and strength represented by its magnetic moment, written as $\vec\mu$. Nuclei with nonzero magnetic moment also have angular momentum, so it can also be thought of as a spinning top with the magnetic axis oriented in the same direction as the rotation axis (see the diagram on the right). When placed in an external magnetic field, which we will call $\vec B_0$, the nucleus will experience a torque pulling it into alignment with $\vec B_0$. However, because the nucleus also has angular momentum it will not rotate directly into alignment, but instead it will precess around $\vec B_0$; behaving like a spinning top that is gradually falling over.

The frequency of this precession is called the Larmor Frequency, given by:

$$f = \frac{\gamma}{2\pi} B_0 \tag{1}$$

$\gamma$ is the gyromagnetic ratio, which is a property of the nucleus. We will be using hydrogen nuclei (the hydrogen in water), which has a gyromagnetic ratio of:

$$\gamma/2\pi = 42.58 \times 10^{6} \ \mathrm{Hz}/\mathrm{T}$$

### 1.2 Hardware

<figure style="float: right;">
<img src="Images/MagnetCore_ilumr.png" width="600" style="display: block;">
<center><figcaption style="width: 300px;">Figure 2: Magnetic Field Directions </figcaption><center>
<img src="Images/MagneticFieldDirection.png" width="600" style="display: block;">
<center><figcaption style="width: 300px;">Figure 3: Conventional Orientation of Fields </figcaption><center>
</figure>

TODO: better image layout?    
    
The precession of the nuclei can be measured with a pickup coil; A current is induced in the coil by the rotating magnetic field of the nuclei in the sample. The plane that the nuclei precess in is perpendicular to the main magnetic field, $\vec B_0$, so the coil should also be perpendicular to $\vec B_0$ to maximise the signal.

The coil measures the component of the net magnetization of the sample, $\vec M$, along its axis.

After waiting some time, $\vec M$ will have completely realigned with $\vec B_0$ and will not induce any signal in the coil. To bring the precession back, the coil is driven to create a magnetic field $\vec B_1$ that oscillates at the Larmor frequency to re-excite the resonance.


> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 1.1: Measure Magnetic Field Strength**
> 
> - Use the Hall probe and compass to find the direction and strength of the magnetic field.
    Make sure to orient the Hall probe in the direction of the field for an accurate reading.
> - Record the field strength in your notebook
> 
> -------------------------------------------------------------------------------------------------------------------------------------------------------

>--- 
> #### **Task 1.2: Calculate Larmor Frequency**
>
> - Using [equation 1](#mjx-eqn-1), calculate the desired frequency of the system and enter your calculated value in Hz below (scientific notation will work, e.g. "5.5e6"):
>
>--- 

In [ ]:
frequency = pn.widgets.FloatInput(name='Frequency (Hz)', start=0, step=1000, width=200)
frequency

## 2. Tuning the RF Probe

<figure style="float: right;">
<img src="Images/CapCircuit.png" width="300" style="display: block;">
<center><figcaption style="width: 300px;">Figure 4: Resonant Circuit</figcaption><center>
</figure>

In order to efficiently generate the $\vec B_1$ field and pickup the signal from the net magnetization $\vec M$, the coil is combined with a capacitor to form a resonant circuit shown in the figure on the right. The resonant frequency of an LC circuit is:

$$f = \frac{1}{2\pi\sqrt{LC}} \tag{2}$$

This frequency needs to be close to the NMR resonant frequency to optimise the transmit and receive efficiency of the probe. Adjusting the *Tune* capacitor on the probe changes the value of $C$. The LC resonator circuit also needs to be impedance matched to the RF transmit amplifier and receive preamplifier, which have a characteristic impedance of 50 Ohms. This is accomplished with a *Match* capacitor.

The *Match* capacitor actually also has some effect on the resonant frequency, and the *Tune* capacitor has some effect on the impedance, so in practice it is an iterative process to optimise these values.

The Wobble tool sweeps through a range of frequencies and measures the amount of power reflected by the probe circuit. The probe will only absorb power close to its resonant frequency, so wobble plot will have a dip at that frequency. At the resonant frequency there will still be some power reflected if there is an impedance mismatch, so the level of the reflected power plot at the bottom of the dip indicates how well matched the probe is.

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 2.1: Tune the Probe**
> - Remove the black plate on top of the ilumr by rotating it anticlockwise.
> - Run the **Wobble** tool below in loop mode. 
> - While the **Wobble** tool is running, adjust the variable capacitors with the provided screwdriver to tune and match the RF coil to your calculated Larmor frequency (Note: the dotted line on the plot corresponds to the desired frequency). **TO DO: show picture of caps. add description of how to know when the probe is tuned enough**
> - Include the dB measurement of your coil tune - this will require a simple calculation. *we need to reword this slightly*
> - Load a sample into the ilumr and observe how "loading" a coil will affect both the tune and match of the coil.
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
from Wobble import WobbleApp # from dashboards-inline directory that was added to sys.path
wobble = WobbleApp(
    override_pars=dict(
        f=frequency, # use value of fequency from the user input
        f_bw=2e6 # fix bandwidth at 2 MHz
    )
)

# TODO: make option in initialiser
wobble.plot.figure.height=400

pn.Column(
    frequency,
    wobble.main(),
    sizing_mode='stretch_width'
)

TO DO: ADD CHART TITLES

NOTE: If using **Run Loop**, click abort before continuing


## 3. Finding the NMR signal

> ---
> #### **Task 3.1: Pulse & Collect** 
> - Use the depth gauge to align the **shim sample** with the 10mm point.
> - Load the **shim sample** into the ilumr.
> - Input your calculated frequency and a starting RF amplitude guess of $0.5$ (half of maximum) into the boxes below.
> - Run the **Pulse & Collect** tool below. The result should be a visible decaying sine wave in the signal plot, and a narrow spike in the spectrum plot.
> - Zoom into the spike in the spectrum plot and read the relative frequency off the frequency axis (use the zoom tools at the top right of the plot).
> - Add the frequency offset you measured to the frequency input and Rerun **Pulse & Collect**. The spike in the spectrum should now be centred at 0 on the relative frequency axis.
> - Record the frequency you have measured in your notebook, and use [equation 1](#mjx-eqn-1) to calculate the field strength from this measured frequency. How does the field strength found this way compare to the field strength measured with the Hall probe in **Task 1.1**?
> ---

In [ ]:
rf_amp = pn.widgets.FloatInput(name='RF Amplitude (fraction of maximum)', start=0, end=1, step=0.1, value=0.5)

from FID import FIDApp # from dashboards-inline directory that was added to sys.path

# setup user shims file
import yaml
SHIM_FILE = os.path.join(LAB_DIR, 'shims.yaml')
DEFAULT_SHIMS = dict(
    shim_x=0,
    shim_y=0,
    shim_z=0,
    shim_z2=0,
    shim_zx=0,
    shim_zy=0,
    shim_xy=0,
    shim_x2y2=0)
if not os.path.isfile(SHIM_FILE):
    with open(SHIM_FILE, 'w') as f:
        yaml.dump(DEFAULT_SHIMS, f)

# create FID experiment using user shims, frequency, and RF amplitude
fid = FIDApp(
    override_par_files=[
        SHIM_FILE
    ],
    override_pars=dict(
        f=frequency, # use value of frequency from the user input
        a_90=rf_amp, # use rf amplitude from the user input
        t_90=32e-6,
        n_scans=1,
        n_samples=2000,
        t_dw=0.5e-6,
        t_acqdelay=50e-6,
        t_end=0.5
    )
)

# TODO: make option in initialiser
fid.plot1.figure.height=400
fid.plot2.figure.height=400

pn.Column(
    pn.Row(frequency, rf_amp),
    fid.main(),
    sizing_mode='stretch_width'
)

### Note: Quadrature Detection and Rotating Reference Frames

At the NMR frequency you measured there would be thousands of cycles in the signal plot above if it were just a measurement of the magnetization along a fixed axis. Instead, to simplify processing and visualization, the signal from the probe is processed using a quadrature detector, which removes a carrier frequency from a signal; this technique is also used in AM radio.

The result is a measurement of the signal in a rotating reference frame. Imagine a camera that is stationary ("lab frame") and one that is rotating at your chosen frequency around the sample ("rotating frame"); the lab frame camera will see the magnetization precessing rapidly at the NMR frequency, however the rotating frame camera will only see the difference between the movement of the magnetization and the movement of the camera. If the rotation frequency of the camera is perfectly set to the NMR frequency, then the measured signal does not oscillate at all!

The real and imaginary parts of the signal are the components along the $y'$ and $x'$ axes in the rotating frame, respectively.

<center><img src="Images/RotatingFrame.png" width="700"></center>
<center><figcaption style="width: 300px;">Figure 6: Rotating Frames </figcaption></center>


> ---
> #### **Task 3.2: Retune the Probe**
> - Now that the correct frequency has been measured, go back to **Task 2.1**, run wobble tool below, and retune the probe if it is poorly tuned to the new frequency. As a rule of thumb, the dip should be within 20kHz of the NMR frequency (dotted line).
> ---


In [ ]:
# Display wobble tool again
wobble.main()

## 4. Optimising the Flip Angle

In the rotating frame, $\vec B_1$ appears like a static field (because it is rotating in the lab frame at the same frequency as the rotating frame). In this case, $\vec B_1$ is always aligned with the $x'$ axis, and exerts a torque on the nuclei. This torque causes the nuclei to precess around the $x'$ axis (while still also precessing around $\vec B_0$). If the $\vec B_1$ field is applied for a short time $t$, the final angle of rotation will be (in radians):

$$\alpha = \gamma B_1 t$$

in radians, where $\gamma$ is the gyromagnetic ratio.

The RF pulse amplitude guess we used resulted in signal, but to maximise the signal it needs to be calibrated to cause a $90^\circ$ rotation of the magnetization from the $z$ axis to the $xy$ plane. Remember that the signal we measure is proportional only to the component of $\vec M$ that is in the $xy$ plane, called $\vec M_{xy}$, and the component along the $z$ axis is invisible to the probe. The correct RF pulse amplitude can be empirically determined by choosing a fixed pulse width and measuring the signal with a range of pulse amplitudes to find the maximum.

<center><img src="Images/SpinAngleDiagram.png" width="700"></center>
<center><figcaption style="width: 300px;">Figure 7: Relationship between pulse amplitude, flip angle and signal </figcaption></center>


TODO: 
- Explain this. Does this relate to the pulse calibration step? Only the component in the x,y plane is measurable by the probe coil
- How flip angle relates to area under pulse
- show spin angles - and the pulse amplitudes diagram
- Add the pulse sequence diagram
- Pulse calibration showing full rotation
- Add signal equation show relationship to flip angle


> ---
> #### **Task 4.1: Pulse Calibration** 
> - TODO
> ---

#### **Step Four: Pulse Calibration**

**TO DO: inline dashboard for pulse calibration. Find max by looking at the curve**

BAD :(

With the system tuned, it is time to determine the RF pulse amplitude necessary to achieve a $90^\circ$ tip of the magnetization. Try changing the RF Amplitude above and observing the effect on the initial value of the real part of the **Signal** plot. Roughly what RF Amplitude maximises the signal?

Open the **Pulse Calibration** dashboard from the **Dashboard Apps** sidebar and run it. With **Pulse Calibration** dashboard tool the pulse amplitude for a given pulse length is found to maximize signal - giving the pulse size for a $90^\circ$ tip. Make sure *soft pulse* is **deselected** and the *pulse width* is $32\mathrm{\mu s}$. When the pulse calibration is finished, the results will be automatically saved to the system global variables, and the amplitude value can also be read off the $x$ axis at the position of the dashed line (Note: the axis tick labels use SI prefixes, so 100m means 0.1).

## 4. Decay - T1,T2,T2*
Basic T1, T2. Diagram with these labelled 

Intro homogeneity - maybe hall probe again



Run FID again with correct pulse amplitude. Maximised inital value. Want to maximis the area under the signal - T2*.
Explain the different decays

Diagrams:
- Homogeneity diagram - arrows for different field strengths
- T1 returning to vertical
- T2 dephasing
- Diagram with T2* decay 




TODO:
Label net magnetization component in xy plane as $\vec M_{xy}$

<!--<center><img src="Images/T1Decay.png" width="400"></center>-->
<center><img src="Images/T2StarPhaseDiagram.png" width="1300"></center>
<center><figcaption style="width: 300px;">Figure 8: Dephasing that causes T2 decay </figcaption></center>

<center><img src="Images/T2StarDecay.png" width="600"></center>
<center><figcaption style="width: 300px;">Figure 9: ADD CAPTION </figcaption></center>

## 5. Correcting B0 Field Homogeneity
revisit title

Once the centre frequency has been established and the system set-up to excite and receive signals of that frequency, the system must be shimmed in order to maximize the homogeneity of the magnetic field. Run the **Autoshim** tool below with the **Coarse** setting to shim the system. The *Coarse* setting will start by resetting all the shims to zero and uses a wide bandwidth, whereas the *Quick* setting loads the saved values and uses a narrow bandwidth; it is suitable for retuning the system once everything is set up. The shims will be saved to `shims.yaml` in your `LAB_DIR`, to be used by the FID experiment above.



> ---
> #### **Task 4.1: Run Autoshim**
> - TO DO 
> ---


In [ ]:
# display frequency and rf_amp again as they are used by Autoshim
pn.Row(frequency, rf_amp)

In [ ]:
from Autoshim import AutoshimApp # from dashboards-inline directory that was added to sys.path

autoshim = AutoshimApp(
    output_dir=LAB_DIR,
    override_pars=dict(
        f=frequency, # use value of frequency from the user input
        a_90=rf_amp, # use rf amplitude from the user input
        t_90=32e-6,
    )
)
autoshim.main()

Explain graph

> ---
> #### **Task 4.2: FID Check with Shim**
> - Rerun the FID experiment using the tool below and make note of how the amplitude and decay of the signal has changed compared to without shims in **Step Three**. With a good shim there should still be significant signal at 5 ms. 
> ---


In [ ]:
# create FID experiment using user shims, frequency, and RF amplitude
fid2 = FIDApp(
    override_par_files=[
        SHIM_FILE
    ],
    override_pars=dict(
        f=frequency, # use value of frequency from the user input
        a_90=rf_amp, # use rf amplitude from the user input
        t_90=32e-6,
        n_scans=1,
        n_samples=10000,
        t_dw=0.5e-6,
        t_acqdelay=50e-6,
        t_end=0.5
    )
)
fid2.main()

## 6. Spin Echo 

Diagrams:

Spin echo pulse sequence 

Spin diagrams showing different phase and 180 degree pulse - show realigning



<center><img src="Images/SpinEchoPulse.png" width="800"></center>
<center><figcaption style="width: 300px;">Figure 10: Spin echo pulse sequence </figcaption></center>

<center><img src="Images/SpinEchoPhase.png" width="1300"></center>
<center><figcaption style="width: 300px;">Figure 11: Effect of 180 degree pulse on phase </figcaption></center>